In [2]:
!pip install tensorflow==2.0.0rc

In [6]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf
import pprint


from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

2.0.0-rc0
sys.version_info(major=3, minor=6, micro=3, releaselevel='final', serial=0)
matplotlib 3.1.1
numpy 1.17.1
pandas 0.25.1
sklearn 0.21.3
tensorflow 2.0.0-rc0
tensorflow_core.keras 2.2.4-tf


In [8]:
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
pprint.pprint(housing.DESCR)
print('>'*90)
pprint.pprint(housing.data[1])
pprint.pprint(housing.target)

URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed (_ssl.c:777)>

In [6]:
# 由上面可以知道，有 8 个特征（8 列）,1 个输出（target），20640 个样本（20640 行），
from sklearn.model_selection import train_test_split

# 切分训练集与测试集
x_train_all, x_test, y_train_all, y_test = train_test_split(
    housing.data, housing.target, random_state = 7
)

# 切分全部训练集为训练集与验证集
x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_all, y_train_all, random_state = 11
)

print(x_train.shape, y_train.shape)
print(x_valid.shape, y_valid.shape)
print(x_test.shape, y_test.shape)

(11610, 8) (11610,)
(3870, 8) (3870,)
(5160, 8) (5160,)


In [ ]:
# 数据预处理---标准化（对同一列：减均值，除方差）

from sklearn.preprocessing import StandardScaler
# 派出对象去处理嘛
scaler = StandardScaler()
# 分别标准化三部分输入
x_train_scaled = scaler.fit_transform(x_train)
x_valid_scaled = scaler.transform(x_valid)
x_test_scaled = scaler.transform(x_test)

In [3]:
# ed

output_dir = "generate_csv"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

# 定义持久化函数（使用 csv 格式保存至指定文件夹output_dir）
def save_to_csv(output_dir, data, name_prefix, header=None, n_parts=10):
    path_format = os.path.join(output_dir, "{}_{:02d}.csv")
    filenames = []

    for file_index, row_indices in enumerate(np.array_split(np.arange(len(data)), n_parts)):  
        # .array_split --- Split an array into multiple sub-arrays.
        part_csv = path_format.format(name_prefix, file_index)
        filenames.append(part_csv)

        with open(part_csv, "wt", encoding="utf-8") as f:
            if header is not None:
                f.write(header + '\n')
            for row_index in row_indices:
                f.write(",".join([repr(col) for col in data[row_index]]))
                # repr() --- Return the canonical string representation of the object.
                f.write('\n')  #不要文件在 csv 文件中加入换行符
    return filenames

train_data = np.c_[x_train_scaled, y_train]
valid_data = np.c_[x_valid_scaled, y_valid]
test_data = np.c_[x_test_scaled, y_test]
# np.c_  --- 按列连接两个矩阵，就是把两矩阵左右相加，要求行数相等，类似于pandas中的merge()
# np.r_  --- 按行连接两个矩阵，就是把两矩阵上下相加，要求列数相等，类似于pandas中的concat()

header_cols = housing.feature_names + ["MidianHouseValue"]
header_str = ",".join(header_cols)

train_filenames = save_to_csv(output_dir, train_data, "train", header_str, n_parts=20)
print("train filenames:")
pprint.pprint(train_filenames)

valid_filenames = save_to_csv(output_dir, valid_data, "valid", header_str, n_parts=10)
print("valid filenames:")
pprint.pprint(valid_filenames)

test_filenames = save_to_csv(output_dir, test_data, "test",  header_str, n_parts=10)
print("test filenames:")
pprint.pprint(test_filenames)

NameError: name 'x_train_scaled' is not defined

In [ ]:
# ed

# 从文件名读取csv 文件，并解析：步骤如下
# 1. filename -> dataset
# 2. read file -> dataset -> datasets -> merge
# 3. parse csv


In [9]:
# 以训练集的文件名为例
filename_dataset = tf.data.Dataset.list_files(train_filenames)
for filename in filename_dataset:
    print(filename)

tf.Tensor(b'generate_csv/train_00.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_11.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_06.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_18.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_17.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_13.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_16.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_08.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_15.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_07.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_09.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_03.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_12.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_14.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_05.csv', shape=(), dtype=string)
tf.Tensor(b'generate_csv/train_01.csv', 

In [10]:
n_readers = 5 
dataset = filename_dataset.interleave(
    lambda filename: tf.data.TextLineDataset(filenames=filename).skip(1),
    # tf.data.TextLineDataset(self, filenames, compression_type=None, buffer_size=None, num_parallel_reads=None)
    # A Dataset : comprising lines from one or more text files.
    # 由于第一行为特征名称，所以 skip（1）
    cycle_length = n_readers
)

for line in dataset.take(10):
    print(line.numpy())
    # 获得标准化后的前 10 个训练集

b'0.8115083791797953,-0.04823952235146133,0.5187339067174729,-0.029386394873127775,-0.034064024638222286,-0.05081594842905086,-0.7157356834231196,0.9162751241885168,2.147'
b'2.51504373119231,1.0731637904355105,0.5574401201546321,-0.17273513019187772,-0.612912610473286,-0.01909156503651574,-0.5710993036045546,-0.027490309606616956,5.00001'
b'0.6303435674178064,1.874166156711919,-0.06713214279531016,-0.12543366804152128,-0.19737553788322462,-0.022722631725889016,-0.692407235065288,0.7265233438487496,2.419'
b'0.04326300977263167,-1.0895425985107923,-0.38878716774583305,-0.10789864528874438,-0.6818663605100649,-0.0723871014747467,-0.8883662012710817,0.8213992340186296,1.426'
b'0.04971034572063198,-0.8492418886278699,-0.06214699417830008,0.17878747064657746,-0.8025354230744277,0.0005066066922077538,0.6466457006743215,-1.1060793768010604,2.286'
b'-0.6906143291679195,-0.1283397589791022,7.0201810347470595,5.624287386169439,-0.2663292879200034,-0.03662080416157129,-0.6457503383496215,1.2058962

In [11]:
# ed

# 使用 api 解析 csv
# tf.io.decode_csv(str, record_defaults)
# 例子
sample_str = '1,2,3,4,5'  # 等价于 csv，即 -- 按行读取（一行），按列分隔（五列）的字符串
record_defaults = [
    tf.constant(0, dtype=tf.int32),
    0,
    np.nan,
    "hello",
    tf.constant([])
]

parsed_fields = tf.io.decode_csv(sample_str, record_defaults=record_defaults)
# tf.io.decode_csv() --- Convert CSV records to tensors. Each column maps to one tensor.
# record_defaults --- 指定生成的 tensor 的类型，上面用[]列表表明对不同列保存为 5 种不同类型
pprint.pprint(parsed_fields)
# 生成了 5 个 tensor

[<tf.Tensor: id=95, shape=(), dtype=int32, numpy=1>,
 <tf.Tensor: id=96, shape=(), dtype=int32, numpy=2>,
 <tf.Tensor: id=97, shape=(), dtype=float32, numpy=3.0>,
 <tf.Tensor: id=98, shape=(), dtype=string, numpy=b'4'>,
 <tf.Tensor: id=99, shape=(), dtype=float32, numpy=5.0>]


In [12]:
def parse_csv_line(line, n_fields = 9):
    record_defaults = [tf.constant(np.nan)] * n_fields
    # np.nan -- 通常用于初始化的格式吗
    parsed_fields = tf.io.decode_csv(line, record_defaults=record_defaults)
    # 从传入的 csv 格式文件、字符串中获得 tensor

    # 再把该 tensor 分开合并为 x、y 矩阵
    x = tf.stack(parsed_fields[0:-1]) # 取第一至倒数第二列合并为一个 x 张量
    y = tf.stack(parsed_fields[-1:])  # 取倒数第一列合并为一个 y 张量
    #tf.stack() --  Stacks a list of tensors into one tensor.(按行或列合并张量)

    return x, y

# 例子
pprint.pprint(parse_csv_line(b'-0.9868720801669367,0.832863080552588,-0.18684708416901633,-0.14888949288707784,-0.4532302419670616,-0.11504995754593579,1.6730974284189664,-0.7465496877362412,1.138',
               n_fields=9))

(<tf.Tensor: id=111, shape=(8,), dtype=float32, numpy=
array([-0.9868721 ,  0.8328631 , -0.18684709, -0.1488895 , -0.45323023,
       -0.11504996,  1.6730974 , -0.74654967], dtype=float32)>,
 <tf.Tensor: id=112, shape=(1,), dtype=float32, numpy=array([1.138], dtype=float32)>)


In [13]:
# 总过程
# 1. filename -> dataset
# 2. read file -> dataset -> datasets -> merge
# 3. parse csv


def csv_reader_dataset(filenames, n_readers=5, batch_size=32, n_parse_threads=5,
                       shuffle_buffer_size=10000):
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.repeat()
    dataset = dataset.interleave(
        lambda filename: tf.data.TextLineDataset(filename).skip(1),
        cycle_length = n_readers
    )
    # 为了达到随机的效果（随机选择样本进行训练）：
    dataset.shuffle(shuffle_buffer_size)
    dataset = dataset.map(map_func=parse_csv_line, num_parallel_calls=n_parse_threads)
    # .map() --- Maps map_func across the elements of this dataset.
    dataset = dataset.batch(batch_size)
    # .batch() --- Combines consecutive elements of this dataset into batches.
    return dataset


train_set = csv_reader_dataset(train_filenames, batch_size=3)
# batch_size=3 --- 一次从硬盘读取 3 个样本到内存，以供后续训练
print(train_set)

for x_batch, y_batch in train_set.take(1):
    print("x:________________")
    pprint.pprint(x_batch)
    print("y:________________")
    pprint.pprint(y_batch)

<DatasetV1Adapter shapes: ((None, 8), (None, 1)), types: (tf.float32, tf.float32)>
x:________________
<tf.Tensor: id=196, shape=(3, 8), dtype=float32, numpy=
array([[ 2.5150437 ,  1.0731637 ,  0.5574401 , -0.17273512, -0.6129126 ,
        -0.01909157, -0.5710993 , -0.02749031],
       [-0.097193  , -1.2497431 ,  0.36232963,  0.02690608,  1.0338118 ,
         0.04588159,  1.3418335 , -1.635387  ],
       [ 0.04326301, -1.0895426 , -0.38878718, -0.10789865, -0.68186635,
        -0.0723871 , -0.8883662 ,  0.8213992 ]], dtype=float32)>
y:________________
<tf.Tensor: id=197, shape=(3, 1), dtype=float32, numpy=
array([[5.00001],
       [1.832  ],
       [1.426  ]], dtype=float32)>


In [ ]:
# 读取三个集的 32 个样本到内存
batch_size = 32
train_set = csv_reader_dataset(train_filenames,
                               batch_size = batch_size)
valid_set = csv_reader_dataset(valid_filenames,
                               batch_size = batch_size)
test_set = csv_reader_dataset(test_filenames,
                              batch_size = batch_size)


In [15]:
# keras.models.Sequential() --- Linear stack of layers. 步骤化的图结构
model = keras.models.Sequential([
                                 keras.layers.Dense(30, activation='relu',input_shape=[8]),
                                 keras.layers.Dense(8, activation='relu'),
                                 keras.layers.Dense(1)
])
# 上面的图结构若只有全连接层，相当于线性回归了
model.compile(loss="mean_squared_error", optimizer="sgd")
callbacks = [keras.callbacks.EarlyStopping(patience=10, min_delta=1e-2)]

history = model.fit(train_set,
                    validation_data = valid_set,
                    steps_per_epoch = 11160 // batch_size,
                    validation_steps = 3870 // batch_size,
                    epochs = 100,
                    callbacks = callbacks
                    )

Train for 348 steps, validate for 120 steps
Epoch 1/100
348/348 [==============================] - 2s 5ms/step - loss: 0.6293 - val_loss: 0.5036
Epoch 2/100
348/348 [==============================] - 1s 4ms/step - loss: 0.6920 - val_loss: 0.4754
Epoch 3/100
348/348 [==============================] - 1s 4ms/step - loss: 0.4577 - val_loss: 0.4384
Epoch 4/100
348/348 [==============================] - 1s 4ms/step - loss: 0.4227 - val_loss: 0.4293
Epoch 5/100
348/348 [==============================] - 1s 3ms/step - loss: 0.4892 - val_loss: 0.4234
Epoch 6/100
348/348 [==============================] - 1s 4ms/step - loss: 0.3925 - val_loss: 0.4035
Epoch 7/100
348/348 [==============================] - 1s 4ms/step - loss: 0.3879 - val_loss: 0.3975
Epoch 8/100
348/348 [==============================] - 1s 4ms/step - loss: 0.3749 - val_loss: 0.3936
Epoch 9/100
348/348 [==============================] - 1s 4ms/step - loss: 0.3745 - val_loss: 0.3839
Epoch 10/100
348/348 [=========================

In [16]:
model.evaluate(test_set, steps=5160 // batch_size)

161/161 [==============================] - 0s 2ms/step - loss: 0.3447


0.3446803264643835